# Conversation context

This notebook shows how to leverage *conversation variables* to keep track of the conversation context beyond message history, so they don't get polluted with context information. Key differentiator between conversation variables and message history is that the former is not visible to the user, and will only keep the latest value set during the conversation.

Typical usage includes:

- the channel where the conversation is happening
- the user's language
- the user's timezone or location
- intermediate results or workflow step tracking

Of course, you can store any other information that you want to keep track of during the conversation.

In [ ]:
%load_ext autoreload
%autoreload 2

In [46]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.agent import Agent
from vanilla_aiagents.workflow import Workflow
from vanilla_aiagents.sequence import Sequence
from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)

In this case we will store the user's language and the channel where the conversation is happening with a "first" agent. Then, we will use a "second" agent to show how to access this information.

In [ ]:
# Telling the agents to set context variables implies calling a pre-defined function call
first = Agent(id="first", llm=llm, description="First agent", system_message = """You are part of an AI process
Your task is to set a context for the next agent to continue the conversation.

DO set context variable "CHANNEL" to "voice" and "LANGUAGE" to "en"

DO respond only "Context set" when you are done.
""")

# Second agent might have its system message extended automatically with the context from the ongoing conversation
# "__context__" is a special marker that will be replaced with the context set by the previous agent
second = Agent(id="second", llm=llm, description="Second agent", system_message = """You are part of an AI process
Your task is to continue the conversation based on the context set by the previous agent.
When asked, you can use variable provide in CONTEXT to generate the response.

--- CONTEXT ---
__context__
""")

In [ ]:
flow = Sequence(id="flow", description="", steps=[first, second], llm=llm)
workflow = Workflow(askable=flow)

In [ ]:
workflow.restart()
workflow.run("Which channel is this conversation on?")
workflow.conversation.messages